In [1]:
import numpy as np
import pandas as pd
import jieba
import jieba.posseg
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import sys,codecs
import matplotlib.pyplot as plt
import plotly.express as px

In [5]:
def dataPrepos(text, stopkey):
    l = []
    pos = ['n', 'nz', 'v', 'vd', 'vn', 'l', 'a', 'd']  # 定义选取的词性
    seg = jieba.posseg.cut(text)  # 分词
    for i in seg:
        if i.word not in stopkey and i.flag in pos:  # 去停用词 + 词性筛选
            l.append(i.word)
    return l

# tf-idf获取文本top10关键词
def getKeywords_tfidf(data,stopkey,topK):
    idList, textList = data['id'], data['text']
    corpus = [] # 将所有文档输出到一个list中，一行就是一个文档
    for index in range(len(idList)):
        text = '%s' % (textList[index]) # 获取文本内容
        text = dataPrepos(text,stopkey) # 文本预处理
        text = " ".join(text) # 连接成字符串，空格分隔
        corpus.append(text)

    # 1、构建词频矩阵，将文本中的词语转换成词频矩阵
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus) # 词频矩阵,a[i][j]:表示j词在第i个文本中的词频
    # 2、统计每个词的tf-idf权值
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(X)
    # 3、获取词袋模型中的关键词
    word = vectorizer.get_feature_names_out()
    # 4、获取tf-idf矩阵，a[i][j]表示j词在i篇文本中的tf-idf权重
    weight = tfidf.toarray()
    # 5、打印词语权重
    ids, keys = [], []
    for i in range(len(weight)):
        print("-------这里输出第", i+1 , u"篇文本的词语tf-idf------")
        ids.append(idList[i])
        df_word,df_weight = [],[] # 当前文章的所有词汇列表、词汇对应权重列表
        for j in range(len(word)):
#             print(word[j],weight[i][j])
            df_word.append(word[j])
            
            df_weight.append(weight[i][j])
        df_word = pd.DataFrame(df_word,columns=['word'])
        df_weight = pd.DataFrame(df_weight,columns=['weight'])
        word_weight = pd.concat([df_word, df_weight], axis=1) # 拼接词汇列表和权重列表
        word_weight = word_weight.sort_values(by="weight",ascending = False) # 按照权重值降序排列
        keyword = np.array(word_weight['word']) # 选择词汇列并转成数组格式
        word_split = [keyword[x] for x in range(0,topK)] # 抽取前topK个词汇作为关键词
        word_split = " ".join(word_split)
        keys.append(word_split)

    result = pd.DataFrame({"id": ids, "key": keys},columns=['id','key'])
    return result


def main():
    # 读取数据集
    datapath = 'data/搜狐新闻.csv'
    df = pd.read_csv(datapath,nrows = 100)
    # 停用词表
    stopkey = [w.strip() for w in open('data/停用词汇总.txt', 'r',encoding = 'utf-8').readlines()]
    # tf-idf关键词抽取
    result = getKeywords_tfidf(df,stopkey,10)
    final_df = pd.merge(df,result,how='left',on='id')
    final_df.to_csv("data/keys_TFIDF.csv",index=False)

if __name__ == '__main__':
    main()
    print('finshed')

-------这里输出第 1 篇文本的词语tf-idf------
-------这里输出第 2 篇文本的词语tf-idf------
-------这里输出第 3 篇文本的词语tf-idf------
-------这里输出第 4 篇文本的词语tf-idf------
-------这里输出第 5 篇文本的词语tf-idf------
-------这里输出第 6 篇文本的词语tf-idf------
-------这里输出第 7 篇文本的词语tf-idf------
-------这里输出第 8 篇文本的词语tf-idf------
-------这里输出第 9 篇文本的词语tf-idf------
-------这里输出第 10 篇文本的词语tf-idf------
-------这里输出第 11 篇文本的词语tf-idf------
-------这里输出第 12 篇文本的词语tf-idf------
-------这里输出第 13 篇文本的词语tf-idf------
-------这里输出第 14 篇文本的词语tf-idf------
-------这里输出第 15 篇文本的词语tf-idf------
-------这里输出第 16 篇文本的词语tf-idf------
-------这里输出第 17 篇文本的词语tf-idf------
-------这里输出第 18 篇文本的词语tf-idf------
-------这里输出第 19 篇文本的词语tf-idf------
-------这里输出第 20 篇文本的词语tf-idf------
-------这里输出第 21 篇文本的词语tf-idf------
-------这里输出第 22 篇文本的词语tf-idf------
-------这里输出第 23 篇文本的词语tf-idf------
-------这里输出第 24 篇文本的词语tf-idf------
-------这里输出第 25 篇文本的词语tf-idf------
-------这里输出第 26 篇文本的词语tf-idf------
-------这里输出第 27 篇文本的词语tf-idf------
-------这里输出第 28 篇文本的词语tf-idf------
-------这里输出第 29 篇文本的词语tf-idf-